In [1]:
#Standard Imports
import os
import sys
import pandas as pd
# from collections import defaultdict
import timeit
from docplex.cp.model import CpoModel

In [2]:
#Custom Imports
sys.path.append('../')
from src.chord import Chord
from src.musical_work_input import MusicalWorkInput
from src.cp_model import CPModel
from src.music_functions import *

In [3]:
# Importing Musical Corpus
musical_work_df = pd.read_csv("../data/sample_input.csv")
musical_corpus = []
for i, title, meter, key, tonality, first_on_beat, melody in musical_work_df.itertuples():
    musical_corpus.append(MusicalWorkInput(title, meter, key, tonality, first_on_beat, [int(x) for x in melody.split(',')]))

In [4]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights_temp.csv")

In [5]:
# Choosing Musical Input
music = musical_corpus[0]
print(music.title, music.key, music.tonality, 
      music.first_on_beat,music.melody, music.reference_note)

Ach bleib' bei unsm Herr Jesu Christ 9 major 2 [37, 37, 40, 37, 33, 35, 37, 38, 37, 35, 33, 33, 35, 37, 37, 35, 35, 33, 33, 33, 37, 35, 37, 33, 30, 32, 33, 35, 37, 33, 33, 33, 35, 37, 37, 35, 35, 33, 33, 33] 24


In [6]:
# Importing Chord Vocabulary
if music.tonality == "major":
    chord_df = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
else:
    chord_df = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab = []
for name, note_intervals in chord_df.itertuples():
    chord_vocab.append(Chord(name, set(int(x) for x in note_intervals.split(','))))

In [7]:
# Defining dictionary of weights for each soft constraint options:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
print(soft_constraint_w_weights)

{'chord progression': 0.1, 'chord repetition': 0.1, 'chord bass repetition': 0.1, 'leap resolution': -1.0, 'melodic movement': 0.1, 'note repetition': 0.1, 'parallel movement': -1.0, 'voice overlap': -1.0, 'adjacent bar chords': 0.1, 'chord spacing': -1.0, 'distinct notes': 0.1, 'incomplete chord': -1.0, 'voice crossing': -1.0, 'voice range': 0.1, 'second inversion': 0.1, 'chord distribution': 0.1}


In [8]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing', 'incomplete chord', 'chord distribution']
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords', 'voice crossing', 'parallel movement', 'chord spacing', 'incomplete chord']
                    else False for x in hard_constraint_options}
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'incomplete chord', 'voice crossing', 'voice range',
                           'second inversion', 'chord distribution']

In [9]:
# Defining penalties for chord progression
if music.tonality == "major":
    penalties_chord_progression = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
else:
    penalties_chord_progression = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)

penalties_chord_progression = dict(penalties_chord_progression.stack())

In [10]:
# Model
cp_model = CPModel("test", musical_corpus[0], chord_vocab, penalties_chord_progression,
                   hard_constraints,
                   soft_constraint_w_weights)

In [ ]:
solution = cp_model.solve()

In [ ]:
cp_model.export_midi()

In [ ]:
actual_solution = ['I', 'I', 'V', 'I', 'IV', 'bVII', 'I', 'IV',
                   'I', 'V', 'vi', 'IV', 'IV', 'I', 'I', 'V',
                   'V', 'I', 'I', 'I', 'I', 'V', 'III', 'vi',
                   'II', 'V', '#iv_dim', 'V', 'III', 'vi', 'IV', 'I',
                   'I', 'vi', 'I', 'V', 'V', 'I', 'I', 'I']
count = 0
for i, predicted, actual in zip(range(40), cp_model.get_solution()['Chords'], actual_solution):
    if predicted != actual:
        print(i, predicted, actual)
    else:
        count += 1
similarity_score = count/40
print(similarity_score)